In [28]:
import numpy as np
from astropy.io import fits
import healpy as hp
import pandas as pd
import matplotlib.path as mplPath
import matplotlib.pyplot as plt
import pickle

### Retrieving the relevant CCD Data

In [29]:
decamCCD = fits.open('../bricks_data/ccds-annotated-decam-dr9.fits')
#mosaicCCD = fits.open('../bricks_data/ccds-annotated-mosaic-dr9.fits')
#bassCCD = fits.open('../bricks_data/ccds-annotated-90prime-dr9.fits')
print(decamCCD[1].columns)


ColDefs(
    name = 'image_filename'; format = '120A'
    name = 'image_hdu'; format = 'I'
    name = 'camera'; format = '5A'
    name = 'expnum'; format = 'K'
    name = 'plver'; format = '8A'
    name = 'procdate'; format = '19A'
    name = 'plprocid'; format = '7A'
    name = 'ccdname'; format = '3A'
    name = 'object'; format = '35A'
    name = 'propid'; format = '10A'
    name = 'filter'; format = '1A'
    name = 'exptime'; format = 'E'
    name = 'mjd_obs'; format = 'D'
    name = 'airmass'; format = 'E'
    name = 'fwhm'; format = 'E'
    name = 'width'; format = 'I'
    name = 'height'; format = 'I'
    name = 'ra_bore'; format = 'D'
    name = 'dec_bore'; format = 'D'
    name = 'crpix1'; format = 'E'
    name = 'crpix2'; format = 'E'
    name = 'crval1'; format = 'D'
    name = 'crval2'; format = 'D'
    name = 'cd1_1'; format = 'E'
    name = 'cd1_2'; format = 'E'
    name = 'cd2_1'; format = 'E'
    name = 'cd2_2'; format = 'E'
    name = 'yshift'; format = 'L'
    name = 

In [30]:
dataDecam = decamCCD[1].data
#dataMosaic = mosaicCCD[1].data
#dataBass = bassCCD[1].data

In [31]:
ra0 = dataDecam.field('ra0')
dec0 = dataDecam.field('dec0')
ra1 = dataDecam.field('ra1')
dec1 = dataDecam.field('dec1')
ra2 = dataDecam.field('ra2')
dec2 = dataDecam.field('dec2')
ra3 = dataDecam.field('ra3')
dec3 = dataDecam.field('dec3')
#Extracting First systematic
#filter = dataDecam.field('filter')

airmass = dataDecam.field('airmass')

In [33]:
def raDec2thetaPhi(ra, dec):
    return (0.5 * np.pi - np.deg2rad(dec)), (np.deg2rad(ra))


NSIDE = 256
NPIX = hp.nside2npix(NSIDE)

### Now, trying to actually get all ccds per pixel
Begin by loading the Dictionary mapping pixels to CCDs. Afterwards, iterating through the ccds for a given pixel.
Depending on the systematics, we have to break systematics down into the different bands.
Airmass is simply averaged across all CCDs, exposure times are calculated on a per CCD basis.


In [34]:
with open('../bricks_data/brickpixel2ccd_256.pickle', 'rb') as f:
    pixel2ccd_dict = pickle.load(f)
    f.close()


### 1. Map every pixel to 2k subpixels

- Get center of subpixels using pix2ang
- Then use theta, phi = raDec2thetaPhi(ra, dec) for all subpixels
- pixel_indices = hp.ang2pix(NSIDE, theta,phi)
- Maps the centers of all subpixels to the 256 pixels

In [9]:
NSIDE_SUB = NSIDE * 8
NPIX_SUB = hp.nside2npix(NSIDE_SUB)
print(NSIDE_SUB, NPIX_SUB)

subpixels = np.arange(0,NPIX_SUB)
print(len(subpixels))

2048 50331648
50331648


In [10]:

subpixel_coords = hp.pix2ang(NSIDE_SUB,subpixels)

In [11]:
#Find which subpixels map to which pixels on the 256 grid

theta_sub= subpixel_coords[0]
phi_sub = subpixel_coords[1]
pixel_indices = hp.ang2pix(NSIDE, theta_sub,phi_sub)
print(len(pixel_indices))

50331648


In [12]:
unique = np.unique(pixel_indices)
print(len(unique))

786432


In [14]:
pixel2subpixel_dict = {}

#Loop through the assigned pixel of every subpixel and fill a dict
for subpixel, pixel in enumerate(pixel_indices):
    if pixel_indices[subpixel] not in pixel2subpixel_dict:
        pixel2subpixel_dict[pixel] = []
    pixel2subpixel_dict[pixel].append(subpixel)

In [35]:
# Use this cell to simply import an existing pixel2subpixel mapping

with open('../bricks_data/pixel2subpixel_256_2048.pickle', 'rb') as f:
    pixel2subpixel_dict = pickle.load(f)
    f.close()


In [37]:
# Verify previous step worked by counting list lenght per pixel

#Every Pixel features: This must equal the len of unique pixel NPIX
pixel_count = len(pixel2subpixel_dict.keys())
print(pixel_count)

#Verifying that the 64 subpixels are approximately associated to each pixel
subpixel_count = 0
min = 200
max = 0
for elem in pixel2subpixel_dict.keys():
    elems_in_pixel = len(pixel2subpixel_dict[elem])
    if elems_in_pixel < min:
        min = elems_in_pixel
    if elems_in_pixel > max:
        max = elems_in_pixel
    subpixel_count += elems_in_pixel

print(min)
print(max)
print(subpixel_count)
print(subpixel_count/pixel_count)



786432
64
64
50331648
64.0


In [38]:

''' THIS CODE IS NEEDED WHEN A NEW PIXEL2SUBPIXEL DICT WAS CREATED
with open('../bricks_data/pixel2subpixel_256_2048.pickle', 'wb') as f:
    pickle.dump(pixel2subpixel_dict, f)
    f.close()
'''

" THIS CODE IS NEEDED WHEN A NEW PIXEL2SUBPIXEL DICT WAS CREATED\nwith open('../bricks_data/pixel2subpixel_256_2048.pickle', 'wb') as f:\n    pickle.dump(pixel2subpixel_dict, f)\n    f.close()\n"

### 2. Map every CCD in pixel to 2K subpixels --> use subpixel center
- Clear out dict of all pixels that are not covered by CCDs --> more efficient
- Map every subpixel to its CCDs --> use same code as for pixel2ccd mapping

In [41]:

pixel_keys = list(pixel2subpixel_dict.keys())
print(len(pixel_keys))
for pixel_key in pixel_keys:
    if pixel_key in pixel2ccd_dict:
        continue
    pixel2subpixel_dict.pop(pixel_key)

pixel_keys = pixel2subpixel_dict.keys()
print(len(pixel_keys))
print(pixel2subpixel_dict[459294])

361367
361367
[29364464, 29372655, 29372656, 29380847, 29380848, 29380849, 29389038, 29389039, 29389040, 29389041, 29397230, 29397231, 29397232, 29397233, 29397234, 29405421, 29405422, 29405423, 29405424, 29405425, 29405426, 29413613, 29413614, 29413615, 29413616, 29413617, 29413618, 29413619, 29421804, 29421805, 29421806, 29421807, 29421808, 29421809, 29421810, 29421811, 29429997, 29429998, 29429999, 29430000, 29430001, 29430002, 29430003, 29438189, 29438190, 29438191, 29438192, 29438193, 29438194, 29446382, 29446383, 29446384, 29446385, 29446386, 29454574, 29454575, 29454576, 29454577, 29462767, 29462768, 29462769, 29470959, 29470960, 29479152]


In [ ]:
subpixel2ccd_dict = {}
count_ex = 0

# In this function, I am first converting every CCD into a given polygon using its corner coordinates
# Afterwards, I am using the ang2vec function and the hp.query_polygon function, to identify every subpixel that features the given CCD
# Then, I am creating a dictionary that lists all CCDs for a given pixel

for no in range(len(ra0)):#range(len(ra0)):
    x_coord_polygon = np.array([ra0[no],ra1[no], ra2[no], ra3[no]])
    y_coord_polygon = np.array([dec0[no],dec1[no], dec2[no], dec3[no]])
    theta, phi = raDec2thetaPhi(x_coord_polygon, y_coord_polygon)

    ccd_vertices_3d = hp.ang2vec(theta=theta, phi=phi)

    # Now, Im getting all healpy pixels that a given CCD covers
    try:
        subpixels_for_ccd = hp.query_polygon(nside=NSIDE_SUB, vertices=ccd_vertices_3d, inclusive=True)
    except:
        count_ex += 1
        continue
    # This will give me the mapping CCD --> Pixel
    # Now, im reverting the mapping
    # For every pixel that a given CCD cuts, enter the CCD into the pixel dictionary
    # Mapping: Pixel --> CCDs
    for pixel in subpixels_for_ccd:
        if pixel not in subpixel2ccd_dict:
            subpixel2ccd_dict[pixel] = []
        subpixel2ccd_dict[pixel].append(no)

    if no % 40000 == 0:
        print(no, count_ex)

# Find out what happens to 2467 CCDs where query polygon throws unknown exception

0 0
40000 0
80000 0
120000 0
160000 0
200000 0
240000 0
280000 0
320000 0
360000 0
400000 0
440000 0
480000 0
520000 0
560000 0
600000 0
640000 0
680000 0
720000 0
760000 0
800000 0
840000 0
880000 0
920000 0
960000 0
1000000 0
1040000 0
1080000 0
1120000 0
1160000 0
1200000 1
1240000 1
1280000 1
1320000 1
1360000 1
1400000 1
1440000 1
1480000 1
1520000 1
1560000 1
1600000 2
1640000 5
1680000 11
1720000 11
1760000 16
1800000 34
1840000 34
1880000 34
1920000 34
1960000 34
2000000 41
2040000 41
2080000 41
2120000 41
2160000 43
2200000 43
2240000 43
2280000 43
2320000 43
2360000 124
2400000 124
2440000 125
2480000 125
2520000 125
2560000 125
2600000 154
2640000 154
2680000 154
2720000 154
2760000 154
2800000 154
2840000 154
2880000 154
2920000 154
2960000 154
3000000 154
3040000 184
3080000 184
3120000 184
3160000 184
3200000 184
3240000 185
3280000 185
3320000 187
3360000 187
3400000 615
3440000 674
3480000 677
3520000 679
3560000 679
3600000 693
3640000 693
3680000 693
3720000 693
37600

In [23]:
#Exporting to Pickle, since this process only needs to be run once
#The filename number (e.g.256) indicates the NSIDE of the Healpy pixels used to generate the pixalisation

with open('../bricks_data/brickpixel2ccd_2048.pickle', 'wb') as f:
    pickle.dump(subpixel2ccd_dict, f)
    f.close()



In [46]:
# Use this cell to simply import an existing subpixel2ccd mapping
"""
with open('../bricks_data/brickpixel2ccd_2048.pickle', 'rb') as f:
    subpixel2ccd_dict = pickle.load(f)
    f.close()

""" 

In [49]:
# Validate the subpixel2ccd_dict is correct

subpixel_keys = subpixel2ccd_dict.keys()
print(len(subpixel_keys))
print(subpixel2ccd_dict[459294])
print(pixel2ccd_dict[459294])


361367
[3844955, 3844960, 3844961, 3844966, 3844967, 4633849, 4633850, 4633855, 4633856, 4633863, 4970525, 4970531, 4970537, 4970591, 4970597, 4970603, 4970604, 5054410, 5054411, 5054416, 5054417, 5054424, 5062518, 5062524, 5062530, 5062531, 5062757, 5062763, 5062769, 5150348, 5150353, 5150354, 5150359, 5150360, 5150828, 5150889, 5150897, 5150902, 5150903, 5150908, 5150909, 5644493, 5760528, 5760534, 5760540, 5777260, 5859216, 5859277, 5914962, 5914963, 5914968, 5914969, 5914976, 6080938, 6080944, 6080950, 6080951]
[3844955, 3844960, 3844961, 3844966, 3844967, 4633849, 4633850, 4633855, 4633856, 4633863, 4970525, 4970531, 4970537, 4970591, 4970597, 4970603, 4970604, 5054410, 5054411, 5054416, 5054417, 5054424, 5062518, 5062524, 5062530, 5062531, 5062757, 5062763, 5062769, 5150348, 5150353, 5150354, 5150359, 5150360, 5150828, 5150889, 5150897, 5150902, 5150903, 5150908, 5150909, 5644493, 5760528, 5760534, 5760540, 5777260, 5859216, 5859277, 5914962, 5914963, 5914968, 5914969, 5914976, 6

### 3. Get systematics value across every subpixel and average
- Iterate through all pixels
- For every pixel, get subpixel from pixel2subpixel dict
- Get systematic value at subpixel
- Add and average for pixel

In [31]:
sample_pixel = 459294

# Find subpixel for the given pixel
subpixels_per_pixel = pixel2subpixel_dict[sample_pixel]
airmass_aggregate = 0
print(len(subpixel2ccd_dict.keys()))

c = 0

# Go through all 64 subpixels in the sample pixel
for subpixel in subpixels_per_pixel:
    # Condition needed in case a subpixel is not covered by CCDs
    if subpixel not in subpixel2ccd_dict.keys():
        c +=1
        print(c)
        continue
    ccds_per_subpixel = subpixel2ccd_dict[subpixel]
    print(ccds_per_subpixel)
    airmass_aggregate += airmass[ccds_per_subpixel].sum()/len(ccds_per_subpixel)

airmass_pixel = airmass_aggregate/len(subpixels_per_pixel)

print("Airmass in pixel", sample_pixel, ":", airmass_pixel)

361367
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
Airmass in pixel 459294 : 0.0
